In [ ]:
import re

import pulp as pl

with open('input.txt') as f:
    lines = f.readlines()
    
class Problem:
    def __init__(self, match: re.Match | None):
        if match is None:
            raise ValueError("Invalid line format")
        groups = match.groups()
        self.leds = [c == '#' for c in groups[0]]
        self.buttons = [tuple(int(x) for x in button[1:-1].split(',') if x) for button in groups[1].strip().split(' ')]
        self.joltage_requirement = [int(x) for x in groups[2].split(',')] 
    
    def __repr__(self):
        return f"Problem(leds={self.leds}, buttons={self.buttons}, joltage_requirement={self.joltage_requirement})"
    
line_regex = re.compile(r'^\[([\.#]+)\] ((?:\([0-9,]+\) )+){([0-9,]+)}$')
problems = [Problem(line_regex.match(line)) for line in lines]

## Part 1
Takes ~3s

In [ ]:
total_presses = []

for i, problem in enumerate(problems):
    print(f"Solving problem {i}: {problem}", end='\r')
    optimize_variables = [pl.LpVariable(f'x{i}', lowBound=0, upBound=1, cat='Integer') for i in range(1, len(problem.buttons) + 1)]
    helper_variables = [pl.LpVariable(f'k{i}', lowBound=0, cat='Integer') for i in range(1, len(problem.leds) + 1)]
    
    optimize_prop = pl.LpProblem(f"Problem_{i}", pl.LpMinimize)
    
    for led_index, led_on in enumerate(problem.leds):
        buttons_toggling_leds = [opt_var for opt_var, button in zip(optimize_variables, problem.buttons) if led_index in button]
        optimize_prop += pl.lpSum(buttons_toggling_leds) == (2 * helper_variables[led_index] + (1 if led_on else 0))
        
    optimize_prop += pl.lpSum(optimize_variables), f"Minimize_Total_Button_Presses_{i}"
    
    optimize_prop.solve()
    
    assert optimize_prop.status == pl.LpStatusOptimal, f"Problem {i} did not solve optimally, status: {pl.LpStatus[optimize_prop.status]}"
    total_presses += [pl.value(optimize_prop.objective)]
    
print("\nTotal button presses across all problems for Part 1:", int(sum(total_presses)))

## Part 2
Takes ~3s

In [ ]:
total_presses = []

for i, problem in enumerate(problems):
    print(f"Solving problem {i}: {problem}", end='\r')
    optimize_variables = [pl.LpVariable(f'x{i}', lowBound=0, cat='Integer') for i in range(1, len(problem.buttons) + 1)]
    
    optimize_prop = pl.LpProblem(f"Problem_{i}", pl.LpMinimize)
    
    for led_index, led_on in enumerate(problem.leds):
        buttons_toggling_leds = [opt_var for opt_var, button in zip(optimize_variables, problem.buttons) if led_index in button]
        optimize_prop += pl.lpSum(buttons_toggling_leds) == problem.joltage_requirement[led_index]
        
    optimize_prop += pl.lpSum(optimize_variables), f"Minimize_Total_Button_Presses_{i}"
    
    optimize_prop.solve()
    
    assert optimize_prop.status == pl.LpStatusOptimal, f"Problem {i} did not solve optimally, status: {pl.LpStatus[optimize_prop.status]}"
    total_presses += [pl.value(optimize_prop.objective)]
    
print("\nTotal button presses across all problems for Part 2:", int(sum(total_presses)))